<a href="https://colab.research.google.com/github/swethasrihari/NLP/blob/main/NLP_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
training_path ='/content/drive/MyDrive/Colab Notebooks/sentiment_train.json'
test_path = '/content/drive/MyDrive/Colab Notebooks/hw2_sentiment_test.json'

In [ ]:
train_data = pd.read_json(training_path, lines=True)
test_data = pd.read_json(test_path, lines= True)

In [ ]:
def extract_required_columns(dataset):
  stars = []
  review_title = []
  for record in dataset.values:
    stars.append(record[3])
    review_title.append(record[5])
  return stars,review_title

stars, review_title = extract_required_columns(train_data)
test_stars, test_review_title = extract_required_columns(test_data)


Loading the data and labels into data frames

In [ ]:
data = {"review_title":list(review_title),
        "stars": stars}
df = pd.DataFrame(data)
testing_data = {"review_title":list(test_review_title),
                "stars":test_stars}
test_df = pd.DataFrame(testing_data)

In [ ]:
pip install Transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.2 MB/s eta 0:00:00


Using the BERT tokenizer for padding and converting the review titles into tokens that the model can understand.

In [ ]:
from transformers.models.bert.modeling_tf_bert import TFBertForTokenClassification
import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForTokenClassification.from_pretrained("bert-base-uncased")

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# tokenizing the review titles
import numpy as np
df['tokenized_review'] = df['review_title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=120, padding='max_length', truncation=True))
print(df.shape)

(80000, 3)


In [ ]:
test_df['tokenized_review'] = test_df['review_title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=120, padding='max_length', truncation=True))
print(test_df.shape)

(2000, 3)


Splitting the dataset into training and validation datasets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df['tokenized_review'], df['stars'], test_size=0.2, random_state=42)

print(len(X_train), len(y_train))
print(len(X_val), len(y_val))

64000 64000
16000 16000


In [ ]:
x_train = np.array([np.array(value) for value in X_train])
x_val = np.array([np.array(value) for value in X_val])
x_test = np.array([np.array(value) for value in test_df['tokenized_review']])


In [ ]:
labels = {1:0, 5:1}
y_train = y_train.map(labels)
y_test = test_df['stars'].map(labels)
y_val = y_val.map(labels)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)


In [ ]:
model.fit(x=x_train, y=y_train, epochs=1, batch_size=32, validation_data=(x_val, y_val))

2000/2000 [==============================] - 1669s 814ms/step - loss: 3.9762 - accuracy: 0.8263 - val_loss: 4.0134 - val_accuracy: 0.8864


In [ ]:
y_pred_labels = np.argmax(y_pred.logits, axis=1)

In [ ]:
report = classification_report(y_test, y_pred_labels)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.80      0.89      1000
           1       0.83      0.99      0.91      1000

    accuracy                           0.90      2000
   macro avg       0.91      0.90      0.90      2000
weighted avg       0.91      0.90      0.90      2000



In [ ]:
from sklearn.metrics import confusion_matrix
print("Confusion Matrix : ")
print(confusion_matrix(y_test, y_pred_labels))

Confusion Matrix : 
[[801 199]
 [  8 992]]


Open AI's GPT-2

In [ ]:
# tokenizing the review titles
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
df['tokenized_review'] = df['review_title'].apply(lambda x: tokenizer.encode(x, max_length=120, truncation=True))
test_df['tokenized_review'] = test_df['review_title'].apply(lambda x: tokenizer.encode(x, max_length=120, truncation=True))

In [ ]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(df['tokenized_review'], df['stars'], test_size=0.2, random_state=42)

In [ ]:
# converting into numpy array and padding to ensure that the sequences have the same length
x_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=120)
x_val = tf.keras.preprocessing.sequence.pad_sequences(X_val, padding='post', maxlen=120)
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_df['tokenized_review'], padding='post', maxlen=120)

In [ ]:
# converting labels to 0 or 1
labels = {1: 0, 5: 1}
y_train = y_train.map(labels)
y_test = test_df['stars'].map(labels)
y_val = y_val.map(labels)

In [ ]:
# Load GPT-2 model and add a classification head on top of the model
model = TFGPT2Model.from_pretrained("gpt2", output_hidden_states=True)
inputs = tf.keras.layers.Input(shape=(120,), dtype=tf.int32)
outputs = model(inputs)['last_hidden_state']
pooled_output = tf.reduce_mean(outputs, axis=1)
output = tf.keras.layers.Dense(1, activation='sigmoid')(pooled_output)
classification_model = tf.keras.models.Model(inputs=inputs, outputs=output)

In [ ]:
# Compile and train the model
classification_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [ ]:
classification_model.fit(x=x_train, y=y_train, epochs=1, batch_size=32, validation_data=(x_val, y_val))
# Predict and evaluate
y_pred_probs = classification_model.predict(x_test)
y_pred_labels = (y_pred_probs > 0.5).astype(int)

report = classification_report(y_test, y_pred_labels)
print(report)

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


63/63 [==============================] - 19s 264ms/step
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      1000
           1       0.96      0.92      0.94      1000

    accuracy                           0.94      2000
   macro avg       0.94      0.94      0.94      2000
weighted avg       0.94      0.94      0.94      2000



In [ ]:
from sklearn.metrics import confusion_matrix
print("Confusion Matrix : ")
print(confusion_matrix(y_test, y_pred_labels))

Confusion Matrix : 
[[959  41]
 [ 76 924]]


## Comparing the performance of the BERT and GPT-2 models

* When you compare the confusion matrix of the two models, I noticed that BERT predicted very little false negatives for each class when compared with GPT-2. But it gave more false possitives. For true possitives, both the models did well for one of the classes.

* I also noticed that BERT produced 99% precesion for 0 and 99% recall for 1 compared to 93% and 92% for the GPT2 model.
This indicates that the BERT model has a low rate of false positives for class 0. 99% recall means the model is very good at capturing the positive instances of class1.

Overall I think both models performed equally but BERT slightly better at classifying even though the accuracy score if only 90%.


## preprocessing, training and challenges

* I have not preprocessed the data like the previous assignments.

* I have used the model's tokenizer to do the preprocessing, tokenizing, padding and truncating.

* I have set the maximum length of the tokens to be 120.

* The entire dataset with 80000 rows split into training and validation sets was used for trainig both the models.

* Initially while training with 10000 rows for BERT, the accuracy was 100%. Since the model was getting overfit, I was playing around with the amount of data for training. And fortunately I saw that feeding more data made some difference. I could reduced the dataset a bit more and try to optimize.

* For the BERT model no classification head was added. But for GPT-2 I had to add a classification head due to mismatch in input shape.

* I was stuck with GPT2 for days throughing me all sorts of error but finally I figured that the problem was with the way I was converting the data into numpy arrays. The following suggestion from chatGPT help with all my errors.
tf.keras.preprocessing.sequence.pad_sequences

* I had also faced "out of memory" error a couple of times which was not related to GPU. I didn't understand that error and how it vanished.

* I had to switch multiple times between my student account and personal account for training the models since I ran out of GPU very quickly.


---



Resources used

https://huggingface.co/bert-base-uncased

https://stackoverflow.com/questions/74914230/how-to-import-transformers-with-tensorflow

https://huggingface.co/bert-base-uncased

ChatGPT

In [ ]:
!jupyter nbconvert --to pdf "/content/drive/MyDrive/Colab Notebooks/NLP_HW3.ipynb"
;

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/NLP_HW3.ipynb to pdf
[NbConvertApp] Writing 59220 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 60781 bytes to /content/drive/MyDrive/Colab Notebooks/NLP_HW3.pdf


''